<a href="https://colab.research.google.com/github/kokami236/osiro1/blob/master/tenngunnhokan1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install open3d


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 例: Driveのマイドライブ直下の Colab Notebooks フォルダにある場合
INPUT_FILENAME = "/content/drive/MyDrive/ジオラマ/熊本城小さいジオラマ2分動画手作業.ply"
OUTPUT_FILENAME = "/content/drive/MyDrive/ジオラマ/output熊本城手作業後"

In [ ]:
import open3d as o3d
import numpy as np

def process_point_cloud(input_file, output_file, num_upsampled_points=100000):
    # ... (ファイルの読み込み、ステップ1, 2のノイズ除去と法線推定は省略) ...
    print(f"1. ファイル {input_file} を読み込み中...")
    pcd = o3d.io.read_point_cloud(input_file)
    if not pcd.has_points():
        print("エラー: 点群データが空です。")
        return

    # ステップ1: ノイズ除去 (pcd_clean の作成)
    print("2. ノイズ除去（統計的外れ値除去）を実行中...")
    cl, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
    pcd_clean = pcd.select_by_index(ind)

    # ステップ2: 法線ベクトルの推定
    print("3. 法線ベクトルの推定を実行中...")
    pcd_clean.estimate_normals(
        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30)
    )
    pcd_clean.orient_normals_consistent_tangent_plane(100)

    # -------------------------------------------------------------
    # 🌟 ステップ2.5: 点群のクロップ（切り出し）処理を追加 🌟
    # -------------------------------------------------------------
    print("3.5. 点群を重心から2mの立方体でクロップ中...")

    # ノイズ除去後の点群の中心座標を取得
    center = pcd_clean.get_center()

    # 一辺2mの立方体を定義 (中心からX,Y,Z軸方向に ±1.0m の範囲)
    min_bound = center - 2.0
    max_bound = center + 2.0
    bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound, max_bound)

    # クロップ処理の実行
    pcd_cropped = pcd_clean.crop(bbox)
    print(f"   クロップ前の点数: {len(pcd_clean.points)}, クロップ後の点数: {len(pcd_cropped.points)}")

    # 以降の処理は、切り出した pcd_cropped に対して行う
    target_pcd = pcd_cropped
    # -------------------------------------------------------------

    # --- ステップ3: サーフェス再構築（ポアソン再構築） ---
    print("4. ポアソンサーフェス再構築を実行中 (欠損補完)...")
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        target_pcd, depth=9 # クロップ後の点群を使用
    )

    # --- ステップ4: メッシュからの点群サンプリング ---
    print(f"5. メッシュから {num_upsampled_points} 点をサンプリング中（補完・高密度化）...")
    pcd_upsampled = mesh.sample_points_uniformly(
        number_of_points=num_upsampled_points
    )
    print(f"   最終的な点群の点数: {len(pcd_upsampled.points)}")
    # --- ステップ4-1: 色情報のコピー (省略せずに再掲) ---
    if target_pcd.has_colors():
        print("5-1. 最近傍探索により元の点群の色をコピー中...")
        pcd_clean_tree = o3d.geometry.KDTreeFlann(target_pcd) # クロップ後の点群を探索元にする

        new_colors = []
        for point in pcd_upsampled.points:
            [k, idx, _] = pcd_clean_tree.search_knn_vector_3d(point, 1)
            if k > 0:
                new_colors.append(target_pcd.colors[idx[0]])
            else:
                new_colors.append([0.5, 0.5, 0.5])

        pcd_upsampled.colors = o3d.utility.Vector3dVector(np.asarray(new_colors))
        print("   色情報のコピー完了。")
    else:
        print("   元の点群に色情報がないため、色コピーをスキップします。")
    # --- ステップ5: 結果の保存と確認 ---
    print(f"6.  結果を {output_file} に保存中...")
    o3d.io.write_point_cloud(output_file, pcd_upsampled)

# ... (実行部分は省略) ...

In [ ]:
# --- 実行部分 ---
if __name__ == "__main__":
    # 【注意】Google Colabで実行する前に、Google DriveのマウントとOpen3Dのインストールが必要です。
    #
    # INPUT_FILENAME と OUTPUT_FILENAME は、あなたのGoogle Drive上の正しいパスです。
    INPUT_FILENAME = "/content/drive/MyDrive/ジオラマ/熊本城小さいジオラマ2分動画手作業.ply"
    OUTPUT_FILENAME = "/content/drive/MyDrive/ジオラマ/output熊本城手作業後.ply"

    # 最終的な点群の目標点数 (例: 10万点)
    TARGET_POINT_COUNT = 100000

    process_point_cloud(
        input_file=INPUT_FILENAME,
        output_file=OUTPUT_FILENAME,
        num_upsampled_points=TARGET_POINT_COUNT
    )

1. ファイル /content/drive/MyDrive/ジオラマ/熊本城小さいジオラマ2分動画手作業.ply を読み込み中...
2. ノイズ除去（統計的外れ値除去）を実行中...
3. 法線ベクトルの推定を実行中...
3.5. 点群を重心から2mの立方体でクロップ中...
   クロップ前の点数: 1047775, クロップ後の点数: 1008684
4. ポアソンサーフェス再構築を実行中 (欠損補完)...
5. メッシュから 100000 点をサンプリング中（補完・高密度化）...
   最終的な点群の点数: 100000
5-1. 最近傍探索により元の点群の色をコピー中...
   色情報のコピー完了。
6.  結果を /content/drive/MyDrive/ジオラマ/output熊本城手作業後.ply に保存中...


In [ ]:
# -*- coding: utf-8 -*-
"""endewakeru.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1AGgzcOXi17YG092zDCbofdVmNUBdmJtO
"""

!pip install open3d

from google.colab import drive
drive.mount('/content/drive')

import open3d as o3d
import numpy as np
import os

# --- ユーザーが設定する項目 ---

# 1. 入力する点群ファイルのパス
input_file = "/content/drive/MyDrive/ジオラマ/熊本城小さいジオラマ2分動画手作業.ply"

# 2. 中心の点から残したい半径 (単位は点群の座標系に依存します)
# この値を大きくすると、より多くの点が残ります。
radius = 3.0

# 3. 保存するファイル名
output_file = "/content/drive/MyDrive/centered_point_cloud.ply"

# --------------------------


# ファイルの存在を確認
if not os.path.exists(input_file):
    print(f"エラー: 入力ファイルが見つかりません: {input_file}")
else:
    # 点群データを読み込む
    pcd = o3d.io.read_point_cloud(input_file)

    if not pcd.has_points():
        print("エラー: 点群の読み込みに失敗したか、点が含まれていません。")
    else:
        print(f"処理前の点の数: {len(pcd.points)}")

        # 1. 点群の重心（中心）を計算
        center = pcd.get_center()
        print(f"計算された中心座標: {center}")

        # 2. 各点が中心からどれだけ離れているか計算
        points = np.asarray(pcd.points)

        # NumPyを使って全点の中心からの距離を高速に計算
        distances = np.linalg.norm(points - center, axis=1)

        # 3. 指定した半径の内側にある点のインデックスを取得
        indices = np.where(distances <= radius)[0]

        # 4. 半径内の点群だけを抽出して新しい点群オブジェクトを作成
        centered_pcd = pcd.select_by_index(indices)

        print(f"半径 {radius} m 内の点の数: {len(centered_pcd.points)}")

        # 5. 結果をファイルに書き出す
        o3d.io.write_point_cloud(output_file, centered_pcd)
        print(f"処理後のファイルを {output_file} に保存しました。")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
処理前の点の数: 1054707
計算された中心座標: [ 0.00759311 -0.18214022  0.08869634]
半径 3.0 m 内の点の数: 1032480
処理後のファイルを /content/drive/MyDrive/centered_point_cloud.ply に保存しました。
